# Preliminaries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
from feature_selection import forwardFeatureSelection

sys.path.append("/Users/paolo/Documents/methods/LinCFA")
from LinCFA import LinCFA

sys.path.append("/Users/paolo/Documents/methods/NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("/Users/paolo/Documents/Droughts/Paolo/regression_NonLinCFA")
from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,FS_with_linearWrapper,compare_methods


In [2]:
def plot_cells(output,selected_colnames, xmin=9, xmax=11, ymin=44, ymax=45.5):
    x = []
    y = []
    colors = cm.rainbow(np.linspace(0,1,len(output)))
    fig, ax = plt.subplots(2)
    ax[0].set_xlim(xmin,xmax)
    ax[1].set_xlim(xmin,xmax)
    ax[0].set_ylim(ymin,ymax)
    ax[1].set_ylim(ymin,ymax)
    for i in range(len(output)): 
        #print(len(output[i]))
        x = []
        y = []
        
        for datum in output[i]:
            x.append(float(datum.split('_')[1]))
            y.append(float(datum.split('_')[2]))
        ax[0].scatter(x,y,color=colors[i])
    
    x = []
    y = []
    col = cm.rainbow(np.linspace(0,1,len(selected_colnames)))
    for i in range(len(selected_colnames)): 
        idx = int(selected_colnames[i].split('_')[-1])
        for datum in output[idx]:
            x.append(float(datum.split('_')[1]))
            y.append(float(datum.split('_')[2]))
        ax[1].scatter(x,y,color=col[i])
    

# Temperature

## NonLinCFA: Wrapper and CMI FS

In [4]:
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    for eps in [0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001]:
        print('####################' + basin + '_' + str(eps) + '####################')
        target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
        actual_path = path_features+basin+'_aggreg.csv'
        
        output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
        
        selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)
    
        print('\nFull model and selected features with wrapper\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
        print('\nFull model and best 5 selected features with wrapper\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
        
        ### CMI FS
        res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
        res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
        selectedFeatures='selectedFeatures'
        print(f'\n{res[selectedFeatures]}\n')
        selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
        print('\nFull model and selected features with CMI FS\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
        print('\nFull model and best 5 selected features with CMI FS\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
        
        print('\n\n\n')

####################Adda_0.01####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 3

Number of features: 92

Number of aggregated features: 3

Number 

Number of features: 92

Number of aggregated features: 9

Number of features: 92

Number of aggregated features: 9

Number of features: 92

Number of aggregated features: 5

Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 3

Number of features: 92

Number of aggregated features: 3



selected columns: ['cyclostationary_mean_tg_1w_3', 'cyclostationary_mean_tg_1w_6', 'cyclostationary_mean_tg_4w_3', 'cyclostationary_mean_tg_4w_2', 'cyclostationary_mean_tg_16w_2', 'cyclostationary_mean_tg_1w_0', 'cyclostationary_mean_tg_12w_3', 'cyclostationary_mean_tg_12w_2', 'cyclostationary_mean_tg_1w_4', 'cyclostationary_mean_tg_8w_1'], 

validation score: 0.3144510709749597, 

number of selected features: 10

Full model and selected features with wrapper

Full aggregate regression train score: 0.2763259415917, test score: -0.3592940302337546
Aggregate regression train score with F

CMI: 0.001406046730677818
CMI: 0.007482981950272416
CMI: 0.00245914855097365
Highest CMI score: 0.007482981950272416
Adding original feature: 13
Highest CMI score: -0.0019303749330424058

[12, 20, 10, 13]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.2751104234741426, test score: -0.3531149628604975
Aggregate regression train score with FS: 0.16472444389673602, test score: 0.15922453049007457

Full model and best 5 selected features with CMI FS

Full aggregate regression train score: 0.2751104234741426, test score: -0.3531149628604975
Aggregate regression train score with FS: 0.16472444389673602, test score: 0.15922453049007457




####################Adda_1e-06####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044 

CMI: 0.009190369240083013
CMI: 0.0019426162795177782
CMI: 0.0030981634182589424
CMI: 0.006925969641063345
CMI: 0.0029264662653806783
CMI: 0.0030719299225818453
CMI: 0.0024262826772420976
CMI: 0.0010753876543792146
CMI: 0.02810098729551408
CMI: 0.024144132254435985
CMI: 0.012342105593410813
CMI: 0.006700675919893262
CMI: 0.003896541867466491
CMI: 0.002893936060894886
CMI: 0.0003093615834881408
CMI: 0.010107600077330414
CMI: 0.0076079727244009465
Highest CMI score: 0.02810098729551408
Adding original feature: 20
CMI: 0.0006002957628437888
CMI: 0.0007055900073457133
CMI: 0.011215973931153894
CMI: 0.0031134702805565573
CMI: 0.0008835514222372554
CMI: 0.0016563959094989533
CMI: 0.002038564531746112
CMI: 0.0070333334317422
CMI: 0.007442580696435469
CMI: 0.006915381204424975
CMI: 0.00266293557764663
Highest CMI score: 0.011215973931153894
Adding original feature: 10
CMI: 0.001406046730677818
CMI: 0.007482981950272416
CMI: 0.00245914855097365
Highest CMI score: 0.007482981950272416
Adding orig

CMI: 0.0071005949324310785
CMI: 0.014349402458092284
CMI: 0.009741118894839224
CMI: 0.0034684045593823404
CMI: 0.0041382060252407316
CMI: 0.0031749311461777147
CMI: 0.007348274089656437
CMI: 0.003673037119061945
CMI: 0.029698452495875008
CMI: 0.017410148728450855
CMI: 0.022354373848918793
Highest CMI score: 0.029698452495875008
Adding original feature: 14
CMI: 0.005092404006788509
CMI: 0.022334754505852705
CMI: 0.01631353913053138
CMI: 0.017458821160292154
CMI: 0.01457034116629291
CMI: 0.0061772529667833975
CMI: 0.007790901939134232
CMI: 0.003711718688890031
Highest CMI score: 0.022334754505852705
Adding original feature: 2
CMI: 0.006332958677283118
CMI: 0.0009995828970427445
Highest CMI score: 0.006332958677283118
Adding original feature: 10
CMI: 0.0011758238990016712
CMI: 0.00042950159352572737
Highest CMI score: 0.0011758238990016712
Adding original feature: 5
Highest CMI score: -0.004053613515285048

[6, 9, 14, 2, 10, 5]


Full model and selected features with CMI FS

Full aggregat

...

CMI: 0.0031884852108088058
CMI: 0.00742972102093209
CMI: 0.0014075255031478495
CMI: 0.02092106047253514
Highest CMI score: 0.02092106047253514
Adding original feature: 19
CMI: 0.0018194462341175888
Highest CMI score: 0.0018194462341175888
Adding original feature: 8
CMI: 0.01219685761214176
CMI: 0.004710272423325834
CMI: 0.002670271237424804
CMI: 0.0012022791578195913
Highest CMI score: 0.01219685761214176
Adding original feature: 2
CMI: 0.002891568081888174
Highest CMI score: 0.002891568081888174
Adding original feature: 9
Highest CMI score: -0.0020358651849282927

[0, 17, 10, 19, 8, 2, 9]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.1960940495792498, test score: 0.2111111680380653
Aggregate regression train score with FS: 0.1387109929248792, test score: 0.12750948098387072

Full model and best 5 selected features with CMI FS

Full aggregate regression train score: 0.1960940495792498, test score: 0.2111111680380653
Aggregate regression train 

CMI: 0.00393336402464349
CMI: 0.0009138881542884864
CMI: 0.005089325428070876
CMI: 0.021070736869099743
Highest CMI score: 0.021070736869099743
Adding original feature: 17
CMI: 0.0016369456093404677
CMI: 0.001462202159827647
CMI: 0.005900217108686212
CMI: 0.0021409710435066343
CMI: 0.0027960117718688615
CMI: 0.00287534048339233
Highest CMI score: 0.005900217108686212
Adding original feature: 10
CMI: 0.0035092708465087885
CMI: 0.0004896622717854848
CMI: 0.0031884852108088058
CMI: 0.00742972102093209
CMI: 0.0014075255031478495
CMI: 0.02092106047253514
Highest CMI score: 0.02092106047253514
Adding original feature: 19
CMI: 0.0018194462341175888
Highest CMI score: 0.0018194462341175888
Adding original feature: 8
CMI: 0.01219685761214176
CMI: 0.004710272423325834
CMI: 0.002670271237424804
CMI: 0.0012022791578195913
Highest CMI score: 0.01219685761214176
Adding original feature: 2
CMI: 0.002891568081888174
Highest CMI score: 0.002891568081888174
Adding original feature: 9
Highest CMI score: 

CMI: 0.0021519894117754124
CMI: 0.004261755704103393
CMI: 0.01703074557484377
Highest CMI score: 0.01703074557484377
Adding original feature: 14
Highest CMI score: -0.003284978520595977

[0, 14]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.29722045315591805, test score: -0.35899980533936904
Aggregate regression train score with FS: 0.1493974710370256, test score: 0.13655955265778963

Full model and best 5 selected features with CMI FS

Full aggregate regression train score: 0.29722045315591805, test score: -0.35899980533936904
Aggregate regression train score with FS: 0.1493974710370256, test score: 0.13655955265778963




####################Ticino_0.0001####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.264043    0.00  2001     1 -1.060146
1    2001-01-13  0.354618    0.39  2001     2 -0.405065
2    2001-01-21  0.427990    0.47  2001     3  0.125603
3    2001-01-29  0.339495    0.

CMI: 0.006111170433267019
CMI: 0.0018826991619217553
CMI: 0.006770294357400078
CMI: 0.0005932074888110872
CMI: 0.007197148248515126
Highest CMI score: 0.007197148248515126
Adding original feature: 31
CMI: 0.0032908908731425768
CMI: 0.00025857606466181404
CMI: 0.00854038002522195
CMI: 0.0016787178216375875
CMI: 0.0012168324322618662
Highest CMI score: 0.00854038002522195
Adding original feature: 29
Highest CMI score: -0.0029899898323454133

[0, 31, 29]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.3257799645829278, test score: -0.7777485994139706
Aggregate regression train score with FS: 0.15212438961858676, test score: 0.1319863053913931

Full model and best 5 selected features with CMI FS

Full aggregate regression train score: 0.3257799645829278, test score: -0.7777485994139706
Aggregate regression train score with FS: 0.15212438961858676, test score: 0.1319863053913931




####################Ticino_1e-06####################
target samples:

----- MI Scores -----
[(0, 0.12204987732720178), (9, 0.11925446005307141), (7, 0.11416770556447081), (4, 0.10884306880186691), (10, 0.10351564273308732), (1, 0.10247714968806099), (2, 0.09864801913428013), (3, 0.09537132963722726), (15, 0.09136322538719073), (5, 0.08550627550379862), (8, 0.0854421903059367), (13, 0.08234913055422238), (6, 0.08052176641110485), (12, 0.074248882183563), (14, 0.05705485126180143), (19, 0.0541002468062679), (47, 0.054079985253061985), (44, 0.04472369125908477), (11, 0.041905160553517715), (43, 0.03689026454853463), (45, 0.03366189924529926), (30, 0.03250106409725059), (22, 0.02626414242665474), (20, 0.021618938276464772), (28, 0.02116908211041102), (36, 0.020105403257539622), (40, 0.01960089649247764), (41, 0.019342078560929805), (18, 0.01933678556601927), (21, 0.018509682298164732), (26, 0.017522539747051622), (32, 0.017472116573358128), (29, 0.015770377000271184), (17, 0.012155116170817758), (34, 0.010554288371655113), (38, 0.009226822199977383), (42, 0.

# precipitation

## NonLinCFA: Wrapper and CMI FS

In [5]:
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    for eps in [0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001]:
        print('####################' + basin + '_' + str(eps) + '####################')
        target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
        actual_path = path_features+basin+'_aggreg.csv'
        
        output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
        
        selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)
    
        print('\nFull model and selected features with wrapper\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
        print('\nFull model and best 5 selected features with wrapper\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
        
        ### CMI FS
        res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
        res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
        selectedFeatures='selectedFeatures'
        print(f'\n{res[selectedFeatures]}\n')
        selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
        print('\nFull model and selected features with CMI FS\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
        print('\nFull model and best 5 selected features with CMI FS\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
        
        print('\n\n\n')

####################Adda_0.01####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number 

----- MI Scores -----
[(5, 0.07876850069640887), (6, 0.07227699333374187), (8, 0.05155755293289637), (14, 0.047334700475274924), (1, 0.032674600261714704), (11, 0.032379547749356), (9, 0.03135657205267133), (7, 0.028114044825573174), (3, 0.027155614933176603), (4, 0.02678637412285656), (12, 0.024841567016598537), (10, 0.018859847479014424), (15, 0.009896244111165313), (0, 0.008842103789515575), (16, 0.005044501014415953), (2, -0.0012011767816253746), (13, -0.007314254626464375)]
Best MI score: 0.07876850069640887
Adding first best original feature: 5
CMI: 0.0008727683687697052
Highest CMI score: 0.0008727683687697052
Adding original feature: 4
CMI: 0.0006471526845642506
CMI: 0.002542563685560606
Highest CMI score: 0.002542563685560606
Adding original feature: 6
Highest CMI score: -0.0019383875311486803

[5, 4, 6]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.09634301218586161, test score: -0.06334107735652794
Aggregate regression train score w

Number of features: 92

Number of aggregated features: 6

Number of features: 92

Number of aggregated features: 3

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 2



selected columns: ['cyclostationary_mean_rr_1w_0', 'cyclostationary_mean_rr_1w_1', 'cyclostationary_mean_rr_8w_0', 'cyclostationary_mean_rr_24w_1', 'cyclostationary_mean_rr_24w_0', 'cyclostationary_mean_rr_1w_2', 'cyclostationary_mean_rr_2', 'cyclostationary_mean_rr_1'], 

validation score: 0.04688561398488367, 

number of selected features: 8

Full model and selected features with wrapper

Full aggregate regression train score: 0.0963519949251147, test score: -0.06422006012486059
Aggregate regression train score with FS: 0.08798698279134065, test score: -0.03905639395399252

Full mode

CMI: 0.02081148406711479
Highest CMI score: 0.02081148406711479
Adding original feature: 7
CMI: 0.01239285237075105
CMI: 0.006595239919215692
CMI: 0.003156499071348476
Highest CMI score: 0.01239285237075105
Adding original feature: 1
CMI: 0.0007126872976723903
CMI: 0.016041334648853406
Highest CMI score: 0.016041334648853406
Adding original feature: 5
Highest CMI score: -0.007464917926079992

[4, 6, 7, 1, 5]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.06046776090974659, test score: -0.49689546315211897
Aggregate regression train score with FS: 0.04845226399075098, test score: -0.2900054316186593

Full model and best 5 selected features with CMI FS

Full aggregate regression train score: 0.06046776090974659, test score: -0.49689546315211897
Aggregate regression train score with FS: 0.04845226399075098, test score: -0.2900054316186593




####################Dora_0.0001####################
target samples:            date      mean  median  yea

Number of features: 44

Number of aggregated features: 2

Number of features: 44

Number of aggregated features: 2

Number of features: 44

Number of aggregated features: 2

Number of features: 44

Number of aggregated features: 1

Number of features: 44

Number of aggregated features: 1

Number of features: 44

Number of aggregated features: 1

Number of features: 44

Number of aggregated features: 1



selected columns: ['cyclostationary_mean_rr_4w_1', 'cyclostationary_mean_rr_24w_0', 'cyclostationary_mean_rr_1w_1', 'cyclostationary_mean_rr_4w_0'], 

validation score: 0.0183925733135335, 

number of selected features: 4

Full model and selected features with wrapper

Full aggregate regression train score: 0.061980749027896964, test score: -0.5882856746668321
Aggregate regression train score with FS: 0.05949861754860575, test score: -0.4874221736967548

Full model and best 5 selected features with wrapper

Full aggregate regression train score: 0.061980749027896964, test score: -0.588

...

----- MI Scores -----
[(22, 0.10540351107087807), (42, 0.10038892525270814), (31, 0.09504862903894257), (30, 0.09271731107050801), (44, 0.09118775336609275), (35, 0.08847082224879566), (32, 0.08806435647689961), (36, 0.08721874816890635), (21, 0.08557105673493774), (33, 0.08071317337658432), (23, 0.07685656829841477), (27, 0.07680037628099362), (38, 0.07558723241040187), (41, 0.07423403064202792), (40, 0.07004942161584964), (43, 0.0697176298548119), (25, 0.06924061086735848), (39, 0.06808964688578253), (26, 0.06759455518696374), (37, 0.06736350651771765), (34, 0.0659752327610231), (18, 0.06065550764201256), (20, 0.05790991825514969), (10, 0.0530564818873951), (13, 0.05220501936022941), (29, 0.048093215687404), (16, 0.046824582611973364), (11, 0.04320436268044518), (8, 0.04309325411575538), (12, 0.04306263516036035), (15, 0.03798035132289578), (7, 0.03780601147876466), (9, 0.03584113873773994), (28, 0.03258338307864268), (24, 0.032189802466501834), (17, 0.031328008625869076), (14, 0.030

CMI: 0.0114595075556424
CMI: 0.0074706762670680715
CMI: 0.00020280677764533683
Highest CMI score: 0.0114595075556424
Adding original feature: 33
CMI: 0.002620638185616836
CMI: 0.00489595806267637
CMI: 0.00665929603583354
CMI: 0.003691678094673606
CMI: 0.0015807948095714341
CMI: 0.010792106627653314
CMI: 0.005967429918506739
Highest CMI score: 0.010792106627653314
Adding original feature: 42
CMI: 0.00016498801652314876
CMI: 0.002753862454835887
CMI: 0.004685761033249414
CMI: 0.000294342137844239
Highest CMI score: 0.004685761033249414
Adding original feature: 35
CMI: 0.004072858259701023
CMI: 0.001322270350667648
CMI: 0.00236739194972424
CMI: 0.0005731723317373982
CMI: 0.0015842368823151265
Highest CMI score: 0.004072858259701023
Adding original feature: 21
CMI: 0.0006032314561283947
Highest CMI score: 0.0006032314561283947
Adding original feature: 26
CMI: 0.0012042611455275187
Highest CMI score: 0.0012042611455275187
Adding original feature: 44
CMI: 0.002994538392136009
CMI: 3.35974846

Number of features: 176

Number of aggregated features: 9

Number of features: 176

Number of aggregated features: 12

Number of features: 176

Number of aggregated features: 9

Number of features: 176

Number of aggregated features: 4

Number of features: 176

Number of aggregated features: 5

Number of features: 176

Number of aggregated features: 4

Number of features: 176

Number of aggregated features: 2



selected columns: ['cyclostationary_mean_rr_12w_4', 'cyclostationary_mean_rr_12w_1', 'cyclostationary_mean_rr_1w_4', 'cyclostationary_mean_rr_24w_1', 'cyclostationary_mean_rr_24w_0', 'cyclostationary_mean_rr_4w_4', 'cyclostationary_mean_rr_5', 'cyclostationary_mean_rr_8', 'cyclostationary_mean_rr_3'], 

validation score: 0.18628916761804504, 

number of selected features: 9

Full model and selected features with wrapper

Full aggregate regression train score: 0.23381518560727543, test score: -0.08514044673007826
Aggregate regression train score with FS: 0.15714826194927212, tes

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 2



selected columns: ['cyclostationary_mean_rr_1w_1', 'cyclostationary_mean_rr_24w_0', 'cyclostationary_mean_rr_24w_1', 'cyclostationary_mean_rr_8w_0'], 

validation score: 0.07157762591114547, 

number of selected features: 4

Full model and selected features with wrapper

Full aggregate regression train score: 0.09010601916264616, test score: -0.03549437728480509
Aggregate regression train score with FS: 0.08437806553071059, test score: -0.03783568017876582

Full model and best 5 selected features with wrapper

Full aggregate regression train score: 0.09010601916264616, test score: -0.0

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 3

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 2



selected columns: ['cyclostationary_mean_rr_1w_2', 'cyclostationary_mean_rr_24w_0', 'cyclostationary_mean_rr_24w_1', 'cyclostationary_mean_rr_8w_0'], 

validation score: 0.07074563119275079, 

number of selected features: 4

Full model and selected features with wrapper

Full aggregate regression train score: 0.09613810100977105, test score: -0.0358872159979402
Aggregate regression train score with FS: 0.08155262228663496, test score: -0.043408815766001974

Full model and best 5 selected features with wrapper

Full aggregate regression train score: 0.09613810100977105, test score: -0.0

# Both

## NonLinCFA: Wrapper and CMI FS

In [6]:
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']

path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    for eps in [0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001]:
        print('####################' + basin + '_' + str(eps) + '####################')
        target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
        actual_path = path_features+basin+'_aggreg.csv'
        
        output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
        
        selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)
    
        print('\nFull model and selected features with wrapper\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
        print('\nFull model and best 5 selected features with wrapper\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
        
        ### CMI FS
        res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
        res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
        selectedFeatures='selectedFeatures'
        print(f'\n{res[selectedFeatures]}\n')
        selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
        print('\nFull model and selected features with CMI FS\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
        print('\nFull model and best 5 selected features with CMI FS\n')
        compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
        
        print('\n\n\n')

####################Adda_0.01####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  2001     6 -0.447894
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.382706    0.40  2009    48 -0.263306
407  2009-12-05  0.409921    0.46  2009    49 -0.082282
408  2009-12-13  0.472087    0.53  2009    50  0.331204
409  2009-12-21  0.324728    0.00  2009    52 -0.648940
410  2009-12-29  0.086512    0.00  2009    53 -2.233412

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 3

Number of features: 92

Number of aggregated features: 3

Number 

CMI: 0.006624388253266861
CMI: 0.0030981634182589424
CMI: 0.006282864570508945
CMI: 0.0029264662653806783
CMI: 0.014122997852305863
CMI: 0.006113992740532287
CMI: 0.0030719299225818453
CMI: 0.0024262826772420976
CMI: 0.0010753876543792146
CMI: 0.02810098729551408
CMI: 0.024144132254435985
CMI: 0.012342105593410813
CMI: 0.006700675919893262
CMI: 0.003896541867466491
CMI: 0.002893936060894886
CMI: 0.0003093615834881408
CMI: 0.010107600077330414
CMI: 0.0076079727244009465
CMI: 0.01908967461476177
CMI: 0.012462386718424756
CMI: 0.009210478531231081
CMI: 0.011094173889947156
CMI: 0.014111781613090957
CMI: 0.03333163300111361
CMI: 0.01961083349465806
CMI: 0.01130387795574253
CMI: 0.0034721342846523173
CMI: 0.0351507953297725
CMI: 0.01716450610747053
CMI: 0.023456973403961967
Highest CMI score: 0.0351507953297725
Adding original feature: 45
CMI: 0.003828758131675769
CMI: 0.001600282747443449
CMI: 0.0006175627037483256
CMI: 0.014636880627741128
CMI: 0.006787718912100005
CMI: 0.0180552489424896

Highest CMI score: 0.0351507953297725
Adding original feature: 48
CMI: 0.002267243705612737
CMI: 0.0006135239010789384
CMI: 0.0006175627037483256
CMI: 0.014636880627741128
CMI: 0.006787718912100005
CMI: 0.01805524894248961
CMI: 0.0035822889386327716
CMI: 0.006195223099898034
CMI: 0.01611337274163549
CMI: 0.011218681553707976
CMI: 0.01283079350722205
CMI: 0.006649389573101183
CMI: 0.007149586040914174
CMI: 0.0011920736225755346
Highest CMI score: 0.01805524894248961
Adding original feature: 31
CMI: 5.998019216102435e-05
CMI: 0.0037261674436335213
CMI: 0.001801404151997893
CMI: 0.0065106627476308465
CMI: 0.00811774848951985
CMI: 0.0012067492602157726
CMI: 0.006772490485793453
CMI: 0.005569954623171253
CMI: 0.008943645325141303
CMI: 0.00353633801928932
CMI: 0.002793006852596397
CMI: 0.02447895278320536
CMI: 0.006127627967195848
CMI: 0.0033716731340191486
CMI: 0.009147456493223627
CMI: 0.012512250642266659
CMI: 0.005441452187868484
CMI: 0.0021601408194882044
Highest CMI score: 0.0244789527

Highest CMI score: 0.0006368486998970246
Adding original feature: 10
Highest CMI score: -0.0008805853863484148

[12, 48, 30, 35, 33, 10]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.31617674450603617, test score: -0.3619537782291766
Aggregate regression train score with FS: 0.16313193053929176, test score: 0.16072899812747943

Full model and best 5 selected features with CMI FS

Full aggregate regression train score: 0.31617674450603617, test score: -0.3619537782291766
Aggregate regression train score with FS: 0.1517060797759482, test score: 0.1861320538978487




####################Adda_1e-06####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.546951
1    2001-01-13  0.380618    0.43  2001     2 -0.277191
2    2001-01-21  0.341985    0.38  2001     3 -0.534156
3    2001-01-29  0.322044    0.35  2001     5 -0.666789
4    2001-02-06  0.354954    0.40  200

Number of features: 92

Number of aggregated features: 9

Number of features: 92

Number of aggregated features: 8

Number of features: 92

Number of aggregated features: 5

Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 3

Number of features: 92

Number of aggregated features: 3

Number of features: 92

Number of aggregated features: 6

Number of features: 92

Number of aggregated features: 3

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 2



selected columns: ['cyclostationary_mean_tg_1w_3', 'cyclostationary_mean_tg_1w_6', 'cyclostationary_mean_rr_1w_0', 'cyclostationary_mean_tg_8w_2', 'cyclostationary_mean_tg_4w_2', 'cyclos

...

CMI: 0.00393336402464349
CMI: 0.0009138881542884864
CMI: 0.005089325428070876
CMI: 0.021070736869099743
CMI: 0.00872883993162045
CMI: 0.001245174350323272
CMI: 0.007295661863607528
CMI: 0.014248538977793629
CMI: 0.04557332226287725
CMI: 0.031317268646530366
CMI: 0.021761831150498123
CMI: 0.004135576782026237
CMI: 0.04331210251896961
CMI: 0.024388167891162854
CMI: 0.03559768231451488
CMI: 0.016170283498654736
CMI: 0.05260495298625695
CMI: 0.048406057010018516
CMI: 0.049127883828628094
CMI: 0.031184850290408628
CMI: 0.03885704600862169
CMI: 0.052357978433310054
CMI: 0.03378177655674794
CMI: 0.03338535681360627
CMI: 0.03519423300169247
CMI: 0.03399012970819451
CMI: 0.03677967234395185
CMI: 0.0374475637196387
CMI: 0.03686297403268349
CMI: 0.07692412183660197
CMI: 0.04127394344193808
Highest CMI score: 0.07692412183660197
Adding original feature: 63
CMI: 0.001981425687852839
CMI: 0.001859334341968083
CMI: 0.01911540667983974
CMI: 0.003378067722109801
CMI: 0.009006558906292594
CMI: 0.0021603

CMI: 0.0008279528914997836
CMI: 0.0005786153412906669
CMI: 0.009644859208873008
CMI: 0.012874415682906842
CMI: 0.011699358985369751
CMI: 0.010146776072194219
CMI: 0.006154072504868907
CMI: 0.004021231249343088
Highest CMI score: 0.012874415682906842
Adding original feature: 55
CMI: 0.004219723533200576
CMI: 0.014341957475529493
CMI: 2.9910661623916424e-05
CMI: 0.007249356998779849
CMI: 0.00983000910443152
CMI: 0.009129453440815144
CMI: 0.0009929056361355493
CMI: 0.004219588744683894
CMI: 0.0050323824151991825
CMI: 0.0022870274962712434
Highest CMI score: 0.014341957475529493
Adding original feature: 3
Highest CMI score: -0.00011784487154076206

[0, 63, 50, 55, 3]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.3937812170237238, test score: -0.009363977388628486
Aggregate regression train score with FS: 0.15080296947652228, test score: 0.11475941341229612

Full model and best 5 selected features with CMI FS

Full aggregate regression train score:

----- MI Scores -----
[(0, 0.1233172834270275), (7, 0.11925446005307141), (5, 0.11429640516876576), (3, 0.10839192158463667), (8, 0.10397460958719837), (1, 0.10247714968806099), (2, 0.09513356575733153), (6, 0.0854421903059367), (11, 0.08419891406662146), (4, 0.08052176641110485), (10, 0.07931849119089665), (14, 0.07712008275236602), (44, 0.06849890021359188), (45, 0.06842462865595438), (12, 0.0624493611733322), (43, 0.061138128729906156), (18, 0.0541002468062679), (41, 0.04978620899883007), (46, 0.04938930727611444), (37, 0.04513908122631465), (25, 0.042000102589973554), (9, 0.041905160553517715), (13, 0.04075164347350076), (40, 0.03375222665168136), (38, 0.03366189924529926), (26, 0.03215149480826468), (42, 0.031252692960708414), (47, 0.02756612860711409), (19, 0.02626414242665474), (24, 0.021788321224037514), (17, 0.021397177752193127), (51, 0.019712095569191318), (50, 0.017996597853011345), (29, 0.017804654115437146), (33, 0.017273982036522778), (48, 0.016090289871205584), (49, 0.0

Number of features: 92

Number of aggregated features: 6

Number of features: 92

Number of aggregated features: 5

Number of features: 92

Number of aggregated features: 5

Number of features: 92

Number of aggregated features: 10

Number of features: 92

Number of aggregated features: 9

Number of features: 92

Number of aggregated features: 9

Number of features: 92

Number of aggregated features: 4

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 3

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 1

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 2

Number of features: 92

Number of aggregated features: 2



selected columns: ['cyclostationary_mean_tg_1w_3', 'cyclostationary_mean_rr_1w_2', 'cyclostationary_mean_tg_24w_1', 'cyclostationary_mean_tg_24w_0', 'cyclostationary_mean_tg_24w_2', 'cy

CMI: 0.006111170433267019
CMI: 0.0018826991619217553
CMI: 0.006770294357400078
CMI: 0.0005932074888110872
CMI: 0.007197148248515126
CMI: 0.015478778926102985
CMI: 0.03149216698693984
CMI: 0.00013272790660487366
CMI: 0.020148285578145894
CMI: 0.038315231778118145
CMI: 0.015320964538409435
CMI: 0.005215143093848337
CMI: 0.0034557536488351442
CMI: 0.004052267538868129
Highest CMI score: 0.038315231778118145
Adding original feature: 53
CMI: 0.006925587303421438
CMI: 0.0005209263988580071
CMI: 0.003246805976692524
CMI: 0.002390725054280146
CMI: 0.0008638489437105823
Highest CMI score: 0.006925587303421438
Adding original feature: 1
Highest CMI score: -0.001424523465250238

[0, 53, 1]


Full model and selected features with CMI FS

Full aggregate regression train score: 0.35314705088143317, test score: -1.1511982682920099
Aggregate regression train score with FS: 0.18752880393696636, test score: 0.1691665398211627

Full model and best 5 selected features with CMI FS

Full aggregate regressio

In [9]:
my_dict={}
my_dict['uno'] = 1
my_dict['due'] = 2
my_dict['tre'] = 3
my_dict.fromkeys(['uno','due'])


{'uno': None, 'due': None}